In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from llama_index import download_loader
from pyprojroot import here

MarkdownReader = download_loader("MarkdownReader")

loader = MarkdownReader()
documents = loader.load_data(file=here() / Path("docs/index.md"))
documents

In [ ]:
PDFReader = download_loader("PDFReader")
loader = PDFReader()
documents = loader.load_data(file=here() / "data/dshiring.pdf")
documents

In [ ]:
GPTRepoReader = download_loader("GPTRepoReader")

loader = GPTRepoReader()
documents = loader.load_data(repo_path=here())

In [ ]:
len(documents)